In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import os
import sys
root_folder = os.path.abspath(os.path.dirname(os.getcwd()))
sys.path.append(root_folder)

dagger_training = True
self_training = False
if self_training:
    dagger_training = True

f = []
f2 = []
f3 = []
unfeas_info = []
log = []
log2 = []
log3 = []
n_iter = 1
for i in range(n_iter):
    if self_training:
        f.append(np.load(root_folder+'/decision_transformer/saved_files/checkpoints/checkpoint_ff_ctgrtg_self_' + str(i) + '/log.npz', allow_pickle=True))
        unfeas_info.append({'i_unfeas_scp_cvx' : f[i]['i_unfeas_scp_cvx'], 'i_unfeas_self' : f[i]['i_unfeas_self']})
    else:
        if dagger_training:
            #f.append(np.load(root_folder+'/decision_transformer/saved_files/checkpoints/1_dagger_exp_art/checkpoint_ff_ctgrtg_art_cl_' + str(i) + '/log.npz', allow_pickle=True))
            f.append(np.load(root_folder+'/decision_transformer/saved_files/checkpoints/checkpoint_ff_time40_100_chunk100R_ctgrtg12000_cl_' + str(i) + '/log.npz', allow_pickle=True))
            #f.append(np.load(root_folder+'/decision_transformer/saved_files/checkpoints/checkpoint_ff_ctgrtg_v02_cl_0/log.npz', allow_pickle=True))
            #unfeas_info.append({'i_unfeas_scp_cvx' : f[i]['i_unfeas_scp_cvx'], 'i_unfeas_self' : f[i]['i_unfeas_self']})
        else:
            f.append(np.load(root_folder+'/decision_transformer/saved_files/checkpoints/checkpoint_ff_time40_100_chunkNone_ctgrtg/log.npz', allow_pickle=True))
            f2.append(np.load(root_folder+'/decision_transformer/saved_files/checkpoints/checkpoint_ff_time40_100_chunk_ctgrtg/log.npz', allow_pickle=True))
            f3.append(np.load(root_folder+'/decision_transformer/saved_files/checkpoints/checkpoint_ff_time40_100_chunk100R_ctgrtg/log.npz', allow_pickle=True))
    log.append(f[i]['log'].item())
    if len(f2) > 0:
        log2.append(f2[i]['log'].item())
    if len(f3) > 0:
        log3.append(f3[i]['log'].item())

In [ ]:
# Overall open-loop evaluation loss
plt.figure()
for i in range(n_iter):
    if dagger_training:
        plt.plot(np.arange(0,51)+i*50,log[i]['loss_ol'])
    else:
        plt.plot(log[i]['loss'])
        if len(log2) > 0:
            plt.plot(log2[i]['loss'])
        if len(log3) > 0:
            plt.plot(log3[i]['loss'])
        plt.yscale('log')

In [ ]:
plt.figure()
for i in range(n_iter):
    plt.plot(np.arange(0,51)+i*50,log[i]['loss_cl'])

In [ ]:
# Open-loop evaluation losses divided by states/actions/time-to-gos
setting = 'cl'
plt.figure()
for i in range(n_iter):
    if dagger_training:
        plt.plot(np.arange(0,51)+i*50,log[i]['loss_state_'+setting])
    else:
        plt.plot(log[i]['loss_state'])
        if len(log2) > 0:
            plt.plot(log2[i]['loss_state'])
        if len(log3) > 0:
            plt.plot(log3[i]['loss_state'])
        plt.yscale('log')

plt.figure()
for i in range(n_iter):
    if dagger_training:
        plt.plot(np.arange(0,51)+i*50,log[i]['loss_action_'+setting])
    else:
        plt.plot(log[i]['loss_action'])
        if len(log2) > 0:
            plt.plot(log2[i]['loss_action'])
        if len(log3) > 0:
            plt.plot(log3[i]['loss_action'])
        plt.yscale('log')

plt.figure()
for i in range(n_iter):
    if dagger_training:
        plt.plot(np.arange(0,51)+i*50,log[i]['loss_ttg_'+setting])
    else:
        plt.yscale('log')
        plt.plot(log[i]['loss_ttg'])
        if len(log2) > 0:
            plt.plot(log2[i]['loss_ttg'])
        if len(log3) > 0:
            plt.plot(log3[i]['loss_ttg'])

In [ ]:
if self_training:
    plt.figure()
    for i in range(n_iter):
        plt.scatter(i,len(unfeas_info[i]['i_unfeas_self']))

In [ ]:
dataset = []
#dataset.append(np.load(root_folder+'/optimization/saved_files/closed_loop/1_dagger_exp_art/dagger_checkpoint_ff_ctgrtg_art.npz'))
dataset.append(np.load(root_folder+'/optimization/saved_files/closed_loop/dagger_checkpoint_ff_time40_100_chunk100R_ctgrtg12000.npz'))
for i in range(n_iter-1):
    #dataset.append(np.load(root_folder+'/optimization/saved_files/closed_loop/1_dagger_exp_art/dagger_checkpoint_ff_ctgrtg_art_cl' + str(i) + '.npz'))
    dataset.append(np.load(root_folder+'/optimization/saved_files/closed_loop/dagger_checkpoint_ff_time40_100_chunk100R_ctgrtg12000_cl' + str(i) + '.npz'))

In [ ]:
dataset[0]

In [ ]:
for i in range(n_iter):
    print('DAGGER_ITERATION', str(i), ': Dataset dimension', len(dataset[i]['dataset_ix']))

In [ ]:
for i in range(n_iter):
    print('DAGGER_ITERATION', str(i), ': Number of cvx infeasible', len(dataset[i]['i_unfeas_cvx']))
    print('DAGGER_ITERATION', str(i), ': Number of closed loop infeasible', len(dataset[i]['i_unfeas_artMPC']))

In [ ]:
dataset[0]['dataset_ix'][dataset[0]['i_unfeas_artMPC'][0]],
dataset[0]['J_cvx'][dataset[0]['i_unfeas_artMPC'][0]]

In [ ]:
j = 1
i_dag = 0
plt.figure()
plt.plot(dataset[i_dag]['target_action'][j,:,0].T)
plt.plot(dataset[i_dag]['context_action'][j,:,0].T)

plt.figure()
plt.plot(dataset[i_dag]['target_action'][j,:,1].T)
plt.plot(dataset[i_dag]['context_action'][j,:,1].T)

plt.figure()
plt.plot(dataset[i_dag]['target_action'][j,:,2].T)
plt.plot(dataset[i_dag]['context_action'][j,:,2].T)

In [ ]:
dataset[i_dag]['i_unfeas_artMPC']

In [ ]:
plt.figure()
ax = plt.axes()
from matplotlib.patches import Rectangle, Circle
from optimization.ff_scenario import table, obs, robot_radius
for i in range(1):
    if dataset[i_dag]['context_ctg'][i,0,0] == 0:
        #ax.plot(datasets[0].data['rtgs'][i,:]*0)
        ax.plot(dataset[i_dag]['context_state'][i,:,0], dataset[i_dag]['context_state'][i,:,1])
        ax.plot(dataset[i_dag]['target_state'][i,:,0], dataset[i_dag]['target_state'][i,:,1])
        #ax.scatter((datasets[i_d].data['goal'][i,:,0]*data_stats['goal_std'][:,0]) + data_stats['goal_mean'][:,0], (datasets[i_d].data['goal'][i,:,1]*data_stats['goal_std'][:,1]) + data_stats['goal_mean'][:,1], marker = '*', color='r')
    '''else:
        ax.plot(datasets[0].data['rtgs'][i,:],'r')'''
ax.add_patch(Rectangle((0,0), table['xy_up'][0], table['xy_up'][1], fc=(0.5,0.5,0.5,0.2), ec='k', label='table', zorder=2.5))
for n_obs in range(obs['radius'].shape[0]):
    label_obs = 'obs' if n_obs == 0 else None
    label_robot = 'robot radius' if n_obs == 0 else None
    ax.add_patch(Circle(obs['position'][n_obs,:], obs['radius'][n_obs], fc='r', label=label_obs, zorder=2.5))
    ax.add_patch(Circle(obs['position'][n_obs,:], obs['radius'][n_obs]+robot_radius, fc='r', alpha=0.2, label=label_robot, zorder=2.5))
    
print('Action loss: ', torch.mean((dataset[i_dag]['context_action'][:9,:,:] - dataset[i_dag]['target_action'][:9,:,:])** 2))
plt.show()


In [ ]:

for i_state in range(6):
    plt.figure()
    plt.plot(dataset[i_dag]['context_state'][j,:,i_state])